In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# pip install xgboost
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
import warnings
warnings.filterwarnings('ignore')

# PREPROCESAMIENTO Y CONVERSIÓN DE DATOS
 
# URL del dataset de Enfermedad Cardíaca de Cleveland
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"

# Nombres de columnas según la documentación del dataset
column_names = [
    'age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
    'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'
]

# Cargar el dataset
try:
    df = pd.read_csv(url, names=column_names, na_values='?')
    print("Dataset cargado exitosamente")
except Exception as e:
    print(f"Error al cargar dataset: {e}")
    # Crear dataset de ejemplo para continuar el ejercicio
    from sklearn.datasets import make_classification
    X, y = make_classification(n_samples=303, n_features=13, random_state=42)
    df = pd.DataFrame(X, columns=column_names[:-1])
    df['target'] = y

print(f"Dimensiones del dataset: {df.shape}")

# Exploración inicial de los datos
print("\nEXPLORACIÓN INICIAL:")
print(f"Primeras 5 filas:")
print(df.head())
print(f"\nInformación del dataset:")
print(df.info())
print(f"\nValores nulos por columna:")
print(df.isnull().sum())

# Manejar valores nulos (si existen)
if df.isnull().sum().sum() > 0:
    print("Eliminando filas con valores nulos")
    df = df.dropna()
    print(f"  Dataset después de limpieza: {df.shape}")

# Preprocesar la variable objetivo
# La columna target original tiene valores 0-4, donde 0 = no enfermedad, 1-4 = enfermedad
print("\nPREPROCESAMIENTO DE VARIABLE OBJETIVO:")
print(f"Valores únicos en target antes: {sorted(df['target'].unique())}")

# Convertir a problema binario: 0 = no enfermedad, 1 = enfermedad
df['target_binary'] = (df['target'] > 0).astype(int)

print(f"• Distribución de clases después de binarización:")
print(df['target_binary'].value_counts())
print(f"   - 0: No enfermedad ({(df['target_binary'] == 0).sum()} muestras)")
print(f"   - 1: Enfermedad cardíaca ({(df['target_binary'] == 1).sum()} muestras)")

# Preparar variables predictoras y objetivo
X = df.drop(['target', 'target_binary'], axis=1)  
y = df['target_binary']  # Target binario

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3,       
    random_state=42,     
    stratify=y          
)

print(f"\nDIVISIÓN DE DATOS:")
print(f"Entrenamiento: {X_train.shape[0]} muestras")
print(f"Prueba: {X_test.shape[0]} muestras")

Dataset cargado exitosamente
Dimensiones del dataset: (303, 14)

EXPLORACIÓN INICIAL:
Primeras 5 filas:
    age  sex   cp  trestbps   chol  fbs  restecg  thalach  exang  oldpeak  \
0  63.0  1.0  1.0     145.0  233.0  1.0      2.0    150.0    0.0      2.3   
1  67.0  1.0  4.0     160.0  286.0  0.0      2.0    108.0    1.0      1.5   
2  67.0  1.0  4.0     120.0  229.0  0.0      2.0    129.0    1.0      2.6   
3  37.0  1.0  3.0     130.0  250.0  0.0      0.0    187.0    0.0      3.5   
4  41.0  0.0  2.0     130.0  204.0  0.0      2.0    172.0    0.0      1.4   

   slope   ca  thal  target  
0    3.0  0.0   6.0       0  
1    2.0  3.0   3.0       2  
2    2.0  2.0   7.0       1  
3    3.0  0.0   3.0       0  
4    1.0  0.0   3.0       0  

Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    float64
 1   

In [5]:
# ENTRENAMIENTO Y EVALUACIÓN DE MODELOS


# Diccionario para almacenar resultados
resultados = {}

# MODELO 1: ÁRBOL DE DECISIÓN INDIVIDUAL
print("\n1. ÁRBOL DE DECISIÓN INDIVIDUAL")

"""
**CARACTERÍSTICAS DEL ÁRBOL INDIVIDUAL:**
• Aprece reglas if-else simples
• Alta interpretabilidad
• Propenso a sobreajuste (overfitting)
• Alta varianza - sensible a pequeños cambios en datos
"""

tree_model = DecisionTreeClassifier(random_state=42)
tree_model.fit(X_train, y_train)
y_pred_tree = tree_model.predict(X_test)

accuracy_tree = accuracy_score(y_test, y_pred_tree)
resultados['Árbol Decisión'] = accuracy_tree

print(f"Árbol de Decisión entrenado")
print(f"   • Precisión (Accuracy): {accuracy_tree:.4f}")
print(f"   • Profundidad del árbol: {tree_model.get_depth()}")
print(f"   • Número de hojas: {tree_model.get_n_leaves()}")

# MODELO 2: RANDOM FOREST (BAGGING)
print("\n2. RANDOM FOREST - BAGGING")

"""
**CONCEPTO DE BAGGING (Bootstrap Aggregating):**

• Entrena MÚLTIPLES árboles en PARALELO
• Cada árbol ve un subconjunto aleatorio de datos (muestreo con reemplazo)
• Cada árbol considera un subconjunto aleatorio de características
• Combina resultados por VOTACIÓN MAYORITARIA
"""

rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
resultados['Random Forest'] = accuracy_rf

print(f"Random Forest entrenado")
print(f"   • Precisión (Accuracy): {accuracy_rf:.4f}")
print(f"   • Número de árboles: {len(rf_model.estimators_)}")
print(f"   • Características por split: {rf_model.max_features}")

# MODELO 3: XGBOOST (BOOSTING)
print("\n3. XGBOOST - BOOSTING")

"""
**CONCEPTO DE BOOSTING (Refuerzo Secuencial):**

• Entrena árboles de forma SECUENCIAL (uno tras otro)
• Cada nuevo árbol se enfoca en corregir errores del anterior
• Asigna mayor peso a muestras mal clasificadas
• Combina resultados por SUMA PONDERADA
"""

xgb_model = XGBClassifier(random_state=42, eval_metric='logloss')
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
resultados['XGBoost'] = accuracy_xgb

print(f"XGBoost entrenado")
print(f"   • Precisión (Accuracy): {accuracy_xgb:.4f}")
print(f"   • Número de árboles (estimators): {xgb_model.n_estimators}")


1. ÁRBOL DE DECISIÓN INDIVIDUAL
Árbol de Decisión entrenado
   • Precisión (Accuracy): 0.7333
   • Profundidad del árbol: 9
   • Número de hojas: 32

2. RANDOM FOREST - BAGGING
Random Forest entrenado
   • Precisión (Accuracy): 0.8556
   • Número de árboles: 100
   • Características por split: sqrt

3. XGBOOST - BOOSTING
XGBoost entrenado
   • Precisión (Accuracy): 0.8111
   • Número de árboles (estimators): None
